<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/lessons2/Creating_a_feature_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создание признакового пространства

### Импорт библиотек

In [29]:
import pandas as pd
import numpy as np

from sklearn import model_selection, preprocessing, linear_model
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import classification_report, accuracy_score

import nltk
from nltk import collocations
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
for i in ['genesis', 'stopwords', 'punkt']:
  nltk.download(i)

import pickle
from string import punctuation
from collections import Counter

from google.colab import drive 
drive.mount('/content/gdrive')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Mounted at /content/gdrive


## Подготавливаем данные

In [30]:
#@title скачиваем данные
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-08 18:42:43--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-08 18:42:43--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc77821db034bb9f19af04373055.dl.dropboxusercontent.com/cd/0/inline/Bm28QVOTBlQRCNItUbAmc-9-yh_RZJytLZ7q1HbuzoDot2WL1nGxQuxn5XAQRun2OL9Q8Ymr-nGQB3APe0Q4p3KSZ8l7p68w6e-CovM3Xd-v-krmfkK2OZkk_PvMuzqjVxD9QyhKEv_pCnhc11meOp9fJBbfHHJPREQCDMUAWSJWMw/file# [following]
--2022-06-08 18:42:43--  https://uc77821db034bb9f19af04373055.dl.dropboxusercontent.com/cd/0/inline/Bm28QVOTBlQRCNItUbAmc-9-yh_RZJytLZ7q1HbuzoDot2WL1nGxQuxn5XAQRun2OL9Q

In [31]:
noise = stopwords.words('russian') + list(punctuation)

In [32]:
pos = pd.read_csv('positive.csv', sep=";",  usecols=[3], names=['text'])
pos['label'] = ['positive'] * len(pos)
neg = pd.read_csv('negative.csv', sep=";",  usecols=[3], names=['text'])
neg['label'] = ['negative'] * len(neg)
df = pos.append(neg)
watch_df = df.iloc[np.r_[0:5, -5:0]]
print(watch_df)

                                                     text     label
0       @first_timee хоть я и школота, но поверь, у на...  positive
1       Да, все-таки он немного похож на него. Но мой ...  positive
2       RT @KatiaCheh: Ну ты идиотка) я испугалась за ...  positive
3       RT @digger2912: "Кто то в углу сидит и погибае...  positive
4       @irina_dyshkant Вот что значит страшилка :D\nН...  positive
111918  Но не каждый хочет что то исправлять:( http://...  negative
111919  скучаю так :-( только @taaannyaaa вправляет мо...  negative
111920          Вот и в школу, в говно это идти уже надо(  negative
111921  RT @_Them__: @LisaBeroud Тауриэль, не грусти :...  negative
111922  Такси везет меня на работу. Раздумываю приплат...  negative


In [33]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [34]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
corpus[:10]

['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

In [35]:
freq_dict = Counter(corpus)
freq_dict_sorted = sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('не', 69267),
 ('и', 54916),
 ('в', 52853),
 ('я', 52506),
 ('RT', 38070),
 ('на', 35715),
 ('http', 32992),
 ('что', 31472),
 ('...', 28773),
 ('с', 27176)]

## Задание 1
Обучить три классификатора и сравнить полученные результаты, оценить какие токены наиболее важные для классификации :
1. на токенах с высокой частотой
2. на токенах со средней частотой
3. на токенах с низкой частотой




In [36]:
high_freq = 10000 # по закону Ципфа в основном используют высокие ранги с 10**7 (http://rcdl.ru/doc/2006/paper_72_v1.pdf), но будем использовать поменьше 10**4, т.к. иначе все попадает в среднюю выборку
small_freq = 1000 # по закону Ципфа в основном используют низкие ранги с 10**5 (http://rcdl.ru/doc/2006/paper_72_v1.pdf), но будем использовать поменьше 10**3, т.к. иначе все попадает в среднюю выборку

high_tokens = []
middle_tokens = []
little_tokens = []

In [37]:
for i in freq_dict_sorted:
  if i[1] > high_freq:
    high_tokens.append(i[0])
  elif i[1] < small_freq:
    middle_tokens.append(i[0])
  else:
    little_tokens.append(i[0])
print(f"Количество токенов с высокой частотой: {len(high_tokens)}.\n" +
      f"Количество токенов со средней частотой: {len(middle_tokens)}.\n" + 
      f"Количество токенов с низкой частотой: {len(little_tokens)}.")

Количество токенов с высокой частотой: 28.
Количество токенов со средней частотой: 359368.
Количество токенов с низкой частотой: 225.


In [38]:
def report_token(baseline, x_train, x_test, y_train, y_test):
  %%time
  bow = baseline.fit_transform(x_train)
  clf = LogisticRegression(random_state=42)
  clf.fit(bow, y_train)
  pred = clf.predict(baseline.transform(x_test))
  print(classification_report(pred, y_test))

In [39]:
#@title Считаем токены с высокой частотой
stop_words = noise + middle_tokens + little_tokens
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words)
report_token(vec, x_train, x_test, y_train, y_test)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 10 µs
              precision    recall  f1-score   support

    negative       0.80      0.57      0.67     38651
    positive       0.43      0.68      0.53     18058

    accuracy                           0.61     56709
   macro avg       0.61      0.63      0.60     56709
weighted avg       0.68      0.61      0.62     56709



In [40]:
#@title Считаем токены для средней частоты
stop_words = noise + high_tokens + little_tokens
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words)
report_token(vec, x_train, x_test, y_train, y_test)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs
              precision    recall  f1-score   support

    negative       0.80      0.74      0.77     29973
    positive       0.73      0.79      0.76     26736

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



In [41]:
#@title Считаем токены для низкой частоты
stop_words = noise + high_tokens + middle_tokens
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words)
report_token(vec, x_train, x_test, y_train, y_test)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
              precision    recall  f1-score   support

    negative       0.45      0.67      0.54     18677
    positive       0.78      0.59      0.68     38032

    accuracy                           0.62     56709
   macro avg       0.62      0.63      0.61     56709
weighted avg       0.67      0.62      0.63     56709



**Вывод:**

Судя по границам, которые задали лучшая точность у токенов получилась на средей частоте.

## Задание 2
Найти фичи с наибольшей значимостью, и вывести их.



### Немного теории.
Т.к. анализируем чат, то явно в чате используются тектовые смайлики и они так же несут смысловую нагрузку на текст, из-за чего точность можно улучшить, если их не будем принимать как стоп-слова. Первое что приходит в голову это улыбающийся смайлик: 
*   )
*   :)
Проверим на них.

In [42]:
pred = ['positive' if (')' in tweet or ':)' in tweet) else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32821
    positive       0.83      1.00      0.91     23888

    accuracy                           0.91     56709
   macro avg       0.91      0.92      0.91     56709
weighted avg       0.93      0.91      0.91     56709



Довольно таки не плохо, на самом деле. Что же проверим какие вообще фичи с наибольшей значимостью у нас используется в переписке.

In [43]:
fichi_token = dict()
for punkt in punctuation:
  pred = ['positive' if punkt in tweet else 'negative' for tweet in x_test]
  fichi_token[punkt] = accuracy_score(pred, y_test)
print(fichi_token)

{'!': 0.5117000828792608, '"': 0.5035884956532473, '#': 0.5020190798638664, '$': 0.4911566065351179, '%': 0.4932903066532649, '&': 0.4915798197816925, "'": 0.4912095081909397, '(': 0.02608051632016082, ')': 0.9124830273854239, '*': 0.5096369183022095, '+': 0.4917914264049798, ',': 0.5038177361618086, '-': 0.5093724100231004, '.': 0.5119116895025481, '/': 0.5434058086018092, ':': 0.5468973178860498, ';': 0.4969934225607928, '<': 0.4912447759614876, '=': 0.49159745366696644, '>': 0.4912447759614876, '?': 0.501842741011127, '@': 0.5665767338517695, '[': 0.4912800437320355, '\\': 0.4912447759614876, ']': 0.49131531150258334, '^': 0.4974342696926414, '_': 0.518065915463154, '`': 0.49089209825600877, '{': 0.4912800437320355, '|': 0.4867305013313583, '}': 0.4912800437320355, '~': 0.49122714207621365}


In [44]:
for ficha in sorted(fichi_token.items(), key=lambda x: x[1], reverse=True)[:10]:
  print(f"{ficha[0]} - {ficha[1]}")

) - 0.9124830273854239
@ - 0.5665767338517695
: - 0.5468973178860498
/ - 0.5434058086018092
_ - 0.518065915463154
. - 0.5119116895025481
! - 0.5117000828792608
* - 0.5096369183022095
- - 0.5093724100231004
, - 0.5038177361618086


## Задание 3
 1. сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)
 2. подобрать оптимальный размер для hashing векторайзера
 3. убедиться что для сетки нет переобучения

In [45]:
stop_words = stopwords.words('russian') + list(punctuation)

vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words)
report_token(vec, x_train, x_test, y_train, y_test)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs
              precision    recall  f1-score   support

    negative       0.80      0.76      0.78     29181
    positive       0.76      0.80      0.78     27528

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



In [46]:
stop_words = stopwords.words('russian')

tf_vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words)
report_token(tf_vec, x_train, x_test, y_train, y_test)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27832
    positive       1.00      1.00      1.00     28877

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [47]:
for param in [2**4, 2**8, 2**10]:
  hash_vec = HashingVectorizer(n_features=param)
  print(f"HashingVectorizer with {param}.\n")
  report_token(hash_vec, x_train, x_test, y_train, y_test)

HashingVectorizer with 16.

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs
              precision    recall  f1-score   support

    negative       0.50      0.54      0.52     26167
    positive       0.58      0.55      0.56     30542

    accuracy                           0.54     56709
   macro avg       0.54      0.54      0.54     56709
weighted avg       0.54      0.54      0.54     56709

HashingVectorizer with 256.

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
              precision    recall  f1-score   support

    negative       0.60      0.61      0.60     27404
    positive       0.63      0.62      0.62     29305

    accuracy                           0.61     56709
   macro avg       0.61      0.61      0.61     56709
weighted avg       0.61      0.61      0.61     56709

HashingVectorizer with 1024.

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
              precision    recall  f1-score   support

    negative 